In [1]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import numpy as np
import pandas as pd
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
doc_df = pd.read_csv('tables/doc_69_76.csv')
id_to_text_list = doc_df['id_to_text'].values
free_text_list = doc_df['text'].values

In [3]:
entity_sent_dict = {}

unwanted_entities = ['DATE','TIME','QUANTITY','ORDINAL','CARDINAL','QUANTITY','MONEY','PERCENT','PERSON']

for text in tqdm(free_text_list):

    doc = nlp(text)

    for sent in doc.sents:
        adj_count = 0
        adj_polarity_sum = 0.0
        adj_subjectivity_sum = 0.0

        sentence_named_entities = []
        entity_chunk = None #(entity,type)

        for token in sent:

            if token.ent_iob_ == 'O':
                if entity_chunk:
                    sentence_named_entities.append(entity_chunk)
                    entity_chunk = None
                if token.pos_ == 'ADJ' and token._.blob.polarity!= 0.0:
                    adj_count += 1
                    adj_polarity_sum += token._.blob.polarity
                    adj_subjectivity_sum += token._.blob.subjectivity
                    
            elif token.ent_iob_ == 'B':
                if entity_chunk:
                    sentence_named_entities.append(entity_chunk)
                    entity_chunk = None
                entity_chunk = (token.text,token.ent_type_)
            else:
                entity_chunk_text = entity_chunk[0]
                entity_chunk_type = entity_chunk[1]
                entity_chunk = (entity_chunk_text+' '+token.text,entity_chunk_type)
        
        if adj_count>0:
            sentence_avg_polarity = adj_polarity_sum/adj_count
            sentence_avg_subj = adj_subjectivity_sum/adj_count

            for temp_ne in sentence_named_entities:
                if temp_ne[1] not in unwanted_entities: # if named entity type is valid
                    if entity_sent_dict.get(temp_ne[0],None) is None:
                        entity_sent_dict[temp_ne[0]] = {'pol':[sentence_avg_polarity],'sub':[sentence_avg_subj]}
                    else:
                        entity_sent_dict[temp_ne[0]]['pol'].append(sentence_avg_polarity)
                        entity_sent_dict[temp_ne[0]]['sub'].append(sentence_avg_subj)

100%|██████████| 247/247 [00:43<00:00,  5.67it/s]


In [4]:
for key in entity_sent_dict:
    entity_sent_dict[key]['cnt'] = len(entity_sent_dict[key]['sub'])
    entity_sent_dict[key]['pol'] = round(np.mean(entity_sent_dict[key]['pol']),2)
    entity_sent_dict[key]['sub'] = round(np.mean(entity_sent_dict[key]['sub']),2)

In [5]:
entity_sent_df = pd.DataFrame(entity_sent_dict).transpose()

In [8]:
entity_sent_df.sort_values(by='cnt',ascending=False)[:60]

,pol,sub,cnt
Greece,0.11,0.48,595.0
Turkey,0.14,0.48,523.0
Greek,0.11,0.49,517.0
Turks,0.09,0.55,438.0
Turkish,0.09,0.49,429.0
US,0.11,0.45,428.0
NATO,0.10,0.46,383.0
U.S.,0.09,0.46,177.0
Athens,0.07,0.50,110.0
Congress,0.05,0.53,110.0
